# Data Parsing and Cleaning

This notebook is built to show the steps of converting the web_parse.py outputs into a consistent dataframe for analysis usage. 

1. Unfortunately having two formats parsed from the chase wiki, means some additional work is needed to produce a consistent format (different parsing needed for the different dataframes)
1. What we also see is a need for additional data cleaning which is performed in this notebook

What do we want as an output?

1. Series, Episode Number, AirDate
1. P1, P2, P3, P4 Selected Offer (Zero for failure), Number of Succesful Players 
1. Team Score, Chaser Score, PlayerWin Boolean, Time Remaining on Clock (if chaser win, zero if not)

Note: The data discussed was parsed 20/11/2020, most scraped data formats expected to evolve with time, and possible modifications will be needed in future running.

In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
old_df = pd.read_csv("data/oldformat_chase.csv")
new_df = pd.read_csv("data/newformat_chase.csv")

# Defining Some Parsing Functions

1. The results column is written in the form "17/17 (0:27)" which is "{Team Score}/{Chaser Score} TimeRemaining". Want this parsed into 4 columns output Team Score/Chaser Score/PlayerWin/TimeRemaining

In [3]:
def parse_players_old(in_df):
    """
    Given an input dataframe, with columns P1, P2, P3, P4, 
    - Evaluate how many players moved on to the final round
    - Parse the columns such that theyre ints
    Returns full dataframe
    """

    for p in ["P1", "P2", "P3", "P4"]:
        old_df[p] = old_df[p].str.replace('£', '').str.replace(',', '').str.replace(' ', '')
        old_df[p] = old_df[p].fillna("-")

    old_df["NPlayers"] = ((old_df["P1"].str.isnumeric()).astype(float) + (old_df["P2"].str.isnumeric()).astype(float) +(old_df["P3"].str.isnumeric()).astype(float) +(old_df["P4"].str.isnumeric()).astype(float))
        
    for p in ["P1", "P2", "P3", "P4"]:
        old_df[p] = old_df[p].replace(['-'], '0.0')
        old_df[p] = old_df[p].fillna(0)
        old_df[p] = old_df[p].astype(float)
        
    old_df["PTotal"] = old_df["P1"] + old_df["P2"] + old_df["P3"] + old_df["P4"]
    
    return old_df


def check_contribution(in_string):
    """
    Given an "contribution" string like "() + £9,000 + £5,000 + £8,000"
    Want to return check whether the format is sensible or not, i.e. 4 entries.
    """
    in_string = in_string.replace(",","").replace("£","").replace("+","").lower()

    # These episodes, labelled lazarus and nominated occur when all players lose, 
    # We parse these separately so just return true for this stage
    if ("lazarus" in in_string) or ("nominated" in in_string):
        return True
    
    splits = in_string.split()
    if len(splits) == 4:
        return True
    else:
        return False
    

def parse_players_new(in_string):
    """
    Given an "contribution" string like "() + £9,000 + £5,000 + £8,000"
    Want to return a dictionary with parsed
    P1, P2, P3, P4, NPlayers, PTotal
    """
    
    out_dict = {}
    in_string = in_string.replace(",","").replace("£","").replace("+","").lower()

    # These episodes, labelled lazarus and nominated occur when all players lose, 
    # But the final round still plays out with a certain nominated player.
    if ("lazarus" in in_string) or ("nominated" in in_string):
        out_dict = {"P1" : 0.0, "P2" : 0.0, "P3" : 0.0, "P4" : 0.0, "PTotal" : 4000.0, "NPlayers" : 0}
        return pd.Series(out_dict)
    
    splits = in_string.split()
    
    if len(splits) != 4:
        print(in_string)
    
    ntotal = 0
    ptotal = 0
    for i in range(4):
        if splits[i] == '()':
            out_dict["P{}".format(i+1)] = 0
        else:
            ptotal += float(splits[i].replace("(","").replace(")",""))
            ntotal +=1
            out_dict["P{}".format(i+1)] = float(splits[i])
    
    out_dict["PTotal"] = ptotal
    out_dict["NPlayers"] = ntotal
    
    return pd.Series(out_dict)

def parse_results_old(in_string):
    """
    Given an input results string "17/17 (0:27)" 
    return a dictionary for PlayerScore, ChaserScore, PlayerWin, RemaningTime
    """
    
    out_dict = {}
    
    in_string = in_string.replace("'","")
    
    out_dict["playerScore"] = int(in_string.split("/")[0])
    out_dict["chaserScore"] = int(in_string.split("/")[1].split(" ")[0].split("(")[0])        
        
    if  out_dict["chaserScore"] ==  out_dict["playerScore"]:
        out_dict["playerWin"] = False
        remainingTimeString = in_string.split(")")[0].split("(")[1]
    else:
        out_dict["playerWin"] = True
        remainingTimeString = "00:00"
             
    date_time = datetime.datetime.strptime(remainingTimeString, "%M:%S")
    a_timedelta = date_time - datetime.datetime(1900, 1, 1)
    out_dict["remainingTime"] = a_timedelta.total_seconds()
        
    return pd.Series(out_dict)

# Initial Checks

After parsing raw html data these are how many rows contained in both the old format and new format dataframes!

In [4]:
print("No. Episodes:\nOld: {}\nNew: {}".format(len(old_df), len(new_df)))

No. Episodes:
Old: 471
New: 1064


Immediately we need to check what is even in these dataframes... oh dear. The inconsistencies in the entries imply missing values, and the Dtype being "object" for most columns implies there are some inconsistencies in the type of data stored in each column

In [5]:
old_df.info()
print("")
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 471 entries, 0 to 470
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Series   471 non-null    int64 
 1   Episode  471 non-null    object
 2   AirDate  471 non-null    object
 3   Chaser   471 non-null    object
 4   P1       469 non-null    object
 5   P2       470 non-null    object
 6   P3       470 non-null    object
 7   P4       470 non-null    object
 8   Result   471 non-null    object
dtypes: int64(1), object(8)
memory usage: 33.2+ KB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1064 entries, 0 to 1063
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Series        1064 non-null   int64 
 1   Episode       1064 non-null   object
 2   AirDate       1040 non-null   object
 3   Chaser        1064 non-null   object
 4   Contribution  1035 non-null   object
 5   Result        1034 non-null   ob

# Old Format File
Let's begin with the old dataframe parsing. First let's check a breakdown of counts for which chaser was playing. 
   1. There is one occurence of misspelling of Anne Hegerty : Let's fix the name
   1. There is a row that just has column titles in it, should be removed!


In [6]:
print(old_df[["Chaser", "Result"]].groupby('Chaser').count().rename(columns={'Result':'Episodes'}))
print()
print(old_df[old_df["Chaser"]=="Anne Hegety"].head())
print()
print(old_df[old_df["Chaser"]=="Chaser"].head())

old_df.loc[old_df['Chaser'] == 'Anne Hegety', 'Chaser'] = "Anne Hegerty"
old_df = old_df[old_df["Chaser"] != "Chaser"]

               Episodes
Chaser                 
Anne Hegerty        109
Anne Hegety           1
Chaser                1
Jenny Ryan           34
Mark Labbett        111
Paul Sinha          110
Shaun Wallace       105

     Series Episode          AirDate       Chaser      P1       P2 P3 P4  \
177       8      28  October 8, 2014  Anne Hegety  £6,000  £30,000  -  -   

           Result  
177  15/15 (0:21)  

     Series Episode  AirDate  Chaser  P1  P2  P3  P4              Result
300      11      Ep  Airdate  Chaser  P1  P2  P3  P4  Final Chase result


Turning episode number actually threw an error before the previous filter.
Now that Episode = "Ep" is fixed, can cast Episode number to an int.

In [7]:
old_df["Episode"] = old_df["Episode"].astype(int)
old_df[["Episode"]].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 470 entries, 0 to 470
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   Episode  470 non-null    int64
dtypes: int64(1)
memory usage: 7.3 KB


What rows can we not convert airdate to pandas datetime? The first is an easy fix, the rest is a lot less clear. Checking various websites I can't find too many details on episodes 147-150 and whether they aired or not. Because of this ambiguity I remove from the dataset. The column is then set to datetime!

In [8]:
old_df[pd.to_datetime(old_df["AirDate"], errors='coerce').isnull()]

old_df.loc[old_df['Episode'] == 78, 'AirDate'] = "14/2/2014"
old_df = old_df[old_df["Episode"] < 146]
old_df[pd.to_datetime(old_df["AirDate"], errors='coerce').isnull()]

old_df["AirDate"] = pd.to_datetime(old_df["AirDate"], errors='coerce')

From the HTML we obtain a column for P1, P2, P3, P4. These are marked with a "-" to signify the contestant failed to get through. Some are left blank (assuming by accident) resulting in NaN entries. Also need to do some string parsing to remove the currency symbol and "," symbols for thousands. This is all performed by parse_players_old function!

In [9]:
old_df = parse_players_old(old_df)

Finally want to parse the "Result" string into separate columns, with a boolean for whether the chaser won, and the remaining time left! All performed with the parse_results_old function. Drop the results column as it's no longer needed. 

In [10]:
old_df[["playerScore", "chaserScore", "playerWin", "remainingTime"]] = old_df["Result"].apply(lambda x: parse_results_old(x))
old_df = old_df.drop(['Result'], axis=1)

Okay everything should be handled, let's just check the head...

In [11]:
old_df.head()

,Series,Episode,AirDate,Chaser,P1,P2,P3,P4,NPlayers,PTotal,playerScore,chaserScore,playerWin,remainingTime
0,7,1,2013-09-02,Anne Hegerty,3000.0,0.0,0.0,3000.0,2.0,6000.0,14,14,False,47.0
1,7,2,2013-09-03,Mark Labbett,7000.0,6000.0,5000.0,42000.0,4.0,60000.0,19,18,True,0.0
2,7,3,2013-09-04,Shaun Wallace,1000.0,0.0,0.0,49000.0,2.0,50000.0,17,17,False,2.0
3,7,4,2013-09-05,Mark Labbett,0.0,30000.0,-3000.0,7000.0,2.0,34000.0,19,19,False,6.0
4,7,5,2013-09-06,Paul Sinha,0.0,0.0,4000.0,5000.0,2.0,9000.0,11,11,False,61.0


# New Format File
Let's start looking at the new dataframe parsing. Let's just perform the same checks.... First let's check a breakdown of counts for which chaser was playing. 
   1. Let's fix the entries noted with "(FC)"
   1. Some blank rows with no info labelled with "..." need to remove. Also those labeled with "Chaser"
   1. As of writing this, Darragh Ennis has only appeared in one episode, will be removed until enough stats exists.


In [12]:
print("Before Cleaning:")
print(new_df[["Chaser", "Result"]].groupby('Chaser').count().rename(columns={'Result':'Episodes'}))

new_df = new_df[new_df["Chaser"] != "..."]
new_df = new_df[new_df["Chaser"] != "Chaser"]
new_df = new_df[new_df["Chaser"] != "Darragh Ennis"]

new_df.loc[new_df['Chaser'] == 'Anne Hegerty (FC)', 'Chaser'] = "Anne Hegerty"
new_df.loc[new_df['Chaser'] == 'Mark Labbett\n(FC)', 'Chaser'] = "Mark Labbett"

print("\nAfter Cleaning:")
print(new_df[["Chaser", "Result"]].groupby('Chaser').count().rename(columns={'Result':'Episodes'}))

Before Cleaning:
                    Episodes
Chaser                      
...                        0
Anne Hegerty             237
Anne Hegerty (FC)          1
Chaser                     4
Darragh Ennis              1
Jenny Ryan               122
Mark Labbett             236
Mark Labbett\n(FC)         1
Paul Sinha               206
Shaun Wallace            226

After Cleaning:
               Episodes
Chaser                 
Anne Hegerty        238
Jenny Ryan          122
Mark Labbett        237
Paul Sinha          206
Shaun Wallace       226


There is one episode with additional notes on the episode number, want to fix this up. Can then cast column to integer. 

In [13]:
new_df[pd.to_numeric(new_df['Episode'], errors='coerce').isnull()]
new_df.loc[new_df['Episode'] == '54\n(1,000th Episode)', 'Episode'] = "54"

new_df["Episode"] = new_df["Episode"].astype(int)

new_df[["Episode"]].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1030 entries, 1 to 1053
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   Episode  1030 non-null   int64
dtypes: int64(1)
memory usage: 16.1 KB


Some episodes seem to have not aired, will filter out episodes with AirDate = "TBD". Then convert to the expected airdate column format.

In [14]:
new_df = new_df[new_df["AirDate"] != "TBD"]
new_df["AirDate"] = pd.to_datetime(new_df["AirDate"], errors='coerce')

There is an episode with a lot of data missing, will remove this

In [15]:
print(new_df[new_df["Result"].isnull()].head())
new_df = new_df[~new_df["Result"].isnull()]

     Series  Episode    AirDate         Chaser     Contribution Result
146       5       65 2012-05-14  Shaun Wallace  £7,000 + £7,000    NaN


Can parse the result column as we did before!

In [16]:
new_df[["playerScore", "chaserScore", "playerWin", "remainingTime"]] = new_df["Result"].apply(lambda x: parse_results_old(str(x)))
new_df = new_df.drop(['Result'], axis=1)

There are some episodes with incomplete definitions for the contribution, implying 3 contestants, 5 contestants.
Should be checked, but for this exercise will just remove these episodes.

In [17]:
print(new_df[~new_df["Contribution"].apply(lambda x: check_contribution(str(x)))][["Series", "Episode", "Contribution"]].head())
new_df = new_df[new_df["Contribution"].apply(lambda x: check_contribution(str(x)))]

     Series  Episode                 Contribution
269       6      122   £6,000 + £3,000 +  -£3,000
728      12       45  £9,000 + £6,000 + (-£3,000)
745      12       62   £9,000 + () + () + () + ()
773      12       90    £2,000 + £5,000 + £43,000


Finally can extract values for the player contribitions, all performed by parse_players_new function. Drop Contribution column as no longer needed!

In [18]:
new_df[["P1", "P2", "P3", "P4", "PTotal", "NPlayers"]] = new_df["Contribution"].apply(lambda x: parse_players_new(str(x)))
new_df = new_df.drop(['Contribution'], axis=1)

Now let's just check things look sensible, the order is off from old dataframe, but we can ignore this in the merging!

In [19]:
new_df.head()

,Series,Episode,AirDate,Chaser,playerScore,chaserScore,playerWin,remainingTime,P1,P2,P3,P4,PTotal,NPlayers
1,2,1,2010-05-24,Shaun Wallace,19,19,False,17.0,9000,2000,0,5000,16000.0,3.0
2,2,2,2010-05-25,Mark Labbett,23,23,False,10.0,9000,9000,9000,1000,28000.0,4.0
3,2,3,2010-05-26,Mark Labbett,14,14,False,35.0,9000,0,9000,0,18000.0,2.0
4,2,4,2010-05-27,Shaun Wallace,25,23,True,0.0,4000,12000,2000,1000,19000.0,4.0
5,2,5,2010-05-28,Anne Hegerty,21,21,False,22.0,8000,0,7000,0,15000.0,2.0


# Merging

Now we have a dataframe for both old and new formats, can just merge them and save the result for use in EDA and Modelling studies

In [20]:
merged_df = pd.concat([old_df, new_df], axis=0)
print(len(old_df), len(new_df), len(merged_df))
merged_df.head()

435 1019 1454


,Series,Episode,AirDate,Chaser,P1,P2,P3,P4,NPlayers,PTotal,playerScore,chaserScore,playerWin,remainingTime
0,7,1,2013-09-02,Anne Hegerty,3000,0,0,3000,2.0,6000.0,14,14,False,47.0
1,7,2,2013-09-03,Mark Labbett,7000,6000,5000,42000,4.0,60000.0,19,18,True,0.0
2,7,3,2013-09-04,Shaun Wallace,1000,0,0,49000,2.0,50000.0,17,17,False,2.0
3,7,4,2013-09-05,Mark Labbett,0,30000,-3000,7000,2.0,34000.0,19,19,False,6.0
4,7,5,2013-09-06,Paul Sinha,0,0,4000,5000,2.0,9000.0,11,11,False,61.0


Now let's be careful, do we have any duplicates? Initially I wanted to track duplicate airdates but this seems to bring on many copies, but they have different details.

In the end decided it best to look for duplicates of both Series and Epiosde... thankfully we have none!

In [21]:
merged_df[merged_df.duplicated(['Series','Episode'], keep="first")].head()

,Series,Episode,AirDate,Chaser,P1,P2,P3,P4,NPlayers,PTotal,playerScore,chaserScore,playerWin,remainingTime


Now to save the dataframe and we are done!

In [22]:
merged_df.to_csv('data/parsed_chase.csv', index=False)

In [23]:
!ls -1 data

newformat_chase.csv
oldformat_chase.csv
parsed_chase.csv


In [24]:
merged_df[merged_df.isnull().any(axis=1)]

,Series,Episode,AirDate,Chaser,P1,P2,P3,P4,NPlayers,PTotal,playerScore,chaserScore,playerWin,remainingTime
407,9,105,NaT,Mark Labbett,5000,6000,6000,6000,4.0,23000.0,21,21,False,24.0


In [34]:
test = merged_df[["P1","P2","P3","P4","PTotal"]]
print(len(test[~test.applymap(np.isreal).all(1)]))
print(len(test[test.applymap(np.isreal).all(1)]))

986
468


In [35]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1454 entries, 0 to 1053
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   P1      1454 non-null   object 
 1   P2      1454 non-null   object 
 2   P3      1454 non-null   object 
 3   P4      1454 non-null   object 
 4   PTotal  1454 non-null   float64
dtypes: float64(1), object(4)
memory usage: 68.2+ KB
